In [1]:
import os, sys, time, glob, random, argparse
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn

# XAutoDL 
from xautodl.config_utils import load_config, dict2config, configure2str
from xautodl.datasets import get_datasets, get_nas_search_loaders
from xautodl.procedures import (
    prepare_seed,
    prepare_logger,
    save_checkpoint,
    copy_checkpoint,
    get_optim_scheduler,
)
from xautodl.utils import get_model_infos, obtain_accuracy
from xautodl.log_utils import AverageMeter, time_string, convert_secs2time
from xautodl.models import get_search_spaces

from custom_models import get_cell_based_tiny_net
from custom_search_cells import NAS201SearchCell as SearchCell
from xautodl.models.cell_searchs.genotypes import Structure

# NB201
from nas_201_api import NASBench201API as API

import scipy.stats as stats

2022-11-07 02:53:51.681924: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
parser = argparse.ArgumentParser("Random search for NAS.")
parser.add_argument("--data_path", type=str, default='../cifar.python', help="The path to dataset")
parser.add_argument("--dataset", type=str, default='cifar10',choices=["cifar10", "cifar100", "ImageNet16-120"], help="Choose between Cifar10/100 and ImageNet-16.")

# channels and number-of-cells
parser.add_argument("--search_space_name", type=str, default='nas-bench-201', help="The search space name.")
parser.add_argument("--config_path", type=str, default='./MY.config', help="The path to the configuration.")
parser.add_argument("--max_nodes", type=int, default=4, help="The maximum number of nodes.")
parser.add_argument("--channel", type=int, default=16, help="The number of channels.")
parser.add_argument("--num_cells", type=int, default=5, help="The number of cells in one stage.")
parser.add_argument("--select_num", type=int, default=100, help="The number of selected architectures to evaluate.")
parser.add_argument("--track_running_stats", type=int, default=0, choices=[0, 1], help="Whether use track_running_stats or not in the BN layer.")
# log
parser.add_argument("--workers", type=int, default=4, help="number of data loading workers")
parser.add_argument("--save_dir", type=str, default='./cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric', help="Folder to save checkpoints and log.")
# parser.add_argument("--arch_nas_dataset", type=str, default='../NAS-Bench-201-v1_1-096897.pth', help="The path to load the architecture dataset (tiny-nas-benchmark).")
parser.add_argument("--arch_nas_dataset", type=str, default=None, help="The path to load the architecture dataset (tiny-nas-benchmark).")
parser.add_argument("--print_freq", type=int, default=200, help="print frequency (default: 200)")
parser.add_argument("--rand_seed", type=int, default=None, help="manual seed")
args = parser.parse_args(args=[])
if args.rand_seed is None or args.rand_seed < 0:
    args.rand_seed = random.randint(1, 100000)

    
print(args.rand_seed)
print(args)
xargs=args

65819
Namespace(arch_nas_dataset=None, channel=16, config_path='./MY.config', data_path='../cifar.python', dataset='cifar10', max_nodes=4, num_cells=5, print_freq=200, rand_seed=65819, save_dir='./cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric', search_space_name='nas-bench-201', select_num=100, track_running_stats=0, workers=4)


In [3]:
assert torch.cuda.is_available(), "CUDA is not available."
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_num_threads(xargs.workers)
prepare_seed(xargs.rand_seed)
logger = prepare_logger(args)

Main Function with logger : Logger(dir=cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric, use-tf=False, writer=None)
Arguments : -------------------------------
arch_nas_dataset : None
channel          : 16
config_path      : ./MY.config
data_path        : ../cifar.python
dataset          : cifar10
max_nodes        : 4
num_cells        : 5
print_freq       : 200
rand_seed        : 65819
save_dir         : ./cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric
search_space_name : nas-bench-201
select_num       : 100
track_running_stats : 0
workers          : 4
Python  Version  : 3.7.13 (default, Mar 29 2022, 02:18:16)  [GCC 7.5.0]
Pillow  Version  : 9.0.1
PyTorch Version  : 1.12.0
cuDNN   Version  : 8302
CUDA available   : True
CUDA GPU numbers : 2
CUDA_VISIBLE_DEVICES : None


In [4]:
train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
config = load_config(xargs.config_path, {"class_num": class_num, "xshape": xshape}, logger)
search_loader, _, valid_loader = get_nas_search_loaders(train_data,
                                                        valid_data,
                                                        xargs.dataset,
                                                        "../configs/nas-benchmark/",
                                                        (config.batch_size, config.test_batch_size),
                                                        xargs.workers)
logger.log("||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}".format(
            xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log("||||||| {:10s} ||||||| Config={:}".format(xargs.dataset, config))

search_space = get_search_spaces("cell", xargs.search_space_name)
model_config = dict2config(
    {
        "name": "RANDOM",
        "C": xargs.channel,
        "N": xargs.num_cells,
        "max_nodes": xargs.max_nodes,
        "num_classes": class_num,
        "space": search_space,
        "affine": False,
        "track_running_stats": bool(xargs.track_running_stats),
    },
    None,
)
search_model = get_cell_based_tiny_net(model_config)

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.parameters(), config)

logger.log("w-optimizer : {:}".format(w_optimizer))
logger.log("w-scheduler : {:}".format(w_scheduler))
logger.log("criterion   : {:}".format(criterion))
# if xargs.arch_nas_dataset is None:
api = None
# else:
#     api = API(xargs.arch_nas_dataset)
logger.log("{:} create API = {:} done".format(time_string(), api))

last_info, model_base_path, model_best_path = (
    logger.path("info"),
    logger.path("model"),
    logger.path("best"),
)
network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()

if last_info.exists():  # automatically resume from previous checkpoint
    logger.log(
        "=> loading checkpoint of the last-info '{:}' start".format(last_info)
    )
    last_info = torch.load(last_info)
    start_epoch = last_info["epoch"]
    checkpoint = torch.load(last_info["last_checkpoint"])
    genotypes = checkpoint["genotypes"]
    valid_accuracies = checkpoint["valid_accuracies"]
    search_model.load_state_dict(checkpoint["search_model"])
    w_scheduler.load_state_dict(checkpoint["w_scheduler"])
    w_optimizer.load_state_dict(checkpoint["w_optimizer"])
    logger.log(
        "=> loading checkpoint of the last-info '{:}' start with {:}-th epoch.".format(
            last_info, start_epoch
        )
    )
else:
    logger.log("=> do not find the last-info file : {:}".format(last_info))
    start_epoch, valid_accuracies, genotypes = 0, {"best": -1}, {}

Files already downloaded and verified
Files already downloaded and verified
./MY.config
Configure(scheduler='cos', LR=0.025, eta_min=0.001, epochs=50, warmup=0, optim='SGD', decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=64, test_batch_size=512, class_num=10, xshape=(1, 3, 32, 32))
||||||| cifar10    ||||||| Search-Loader-Num=391, Valid-Loader-Num=49, batch size=64
||||||| cifar10    ||||||| Config=Configure(scheduler='cos', LR=0.025, eta_min=0.001, epochs=50, warmup=0, optim='SGD', decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=64, test_batch_size=512, class_num=10, xshape=(1, 3, 32, 32))
w-optimizer : SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    initial_lr: 0.025
    lr: 0.025
    maximize: False
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005
)
w-scheduler : CosineAnnealingLR(warmup=0, max-epoch=50, current::epoch=0, iter=0.00, type=cosine, T-max=50, eta-min=0.001)
criterion   : CrossEntropyLoss(

In [5]:
def acc_confidence_robustness_metrics(network, inputs, targets):
    with torch.no_grad():
        # accuracy
        network.train()
        _, logits = network(inputs)
        val_top1, val_top5 = obtain_accuracy(logits.data, targets.data, topk=(1, 5))
        acc = val_top1
        
        return acc.item()
        
#         # confidence
#         prob = torch.nn.functional.softmax(logits, dim=1)
#         one_hot_idx = torch.nn.functional.one_hot(targets)
#         confidence = (prob[one_hot_idx==1].sum()) / inputs.size(0) * 100 # in percent
        
#         # sensitivity
#         _, noisy_logits = network(inputs + torch.randn_like(inputs)*0.1)
#         kl_loss = torch.nn.KLDivLoss(reduction="batchmean")
#         sensitivity = kl_loss(torch.nn.functional.log_softmax(noisy_logits, dim=1), torch.nn.functional.softmax(logits, dim=1))
        
#         # robustness
#         original_weights = deepcopy(network.state_dict())
#         for m in network.modules():
#             if isinstance(m, SearchCell):
#                 for p in m.parameters():
#                     p.add_(torch.randn_like(p) * p.std()*0.3)
            
#         _, noisy_logits = network(inputs)
#         kl_loss = torch.nn.KLDivLoss(reduction="batchmean")
#         robustness = -kl_loss(torch.nn.functional.log_softmax(noisy_logits, dim=1), torch.nn.functional.softmax(logits, dim=1))
#         network.load_state_dict(original_weights)
                
#         return acc.item(), confidence.item(), sensitivity.item(), robustness.item()
    
def step_sim_metric(network, criterion, inputs, targets):
#     inputs, targets = inputs[:64], targets[:64] # smaller batches
    original_dict = deepcopy(network.state_dict())
    optim_large_step = torch.optim.SGD(network.parameters(), lr=0.025)
    
    # single large step
    network.train()
    optim_large_step.zero_grad()
    _, logits = network(inputs)
    base_loss = criterion(logits, targets)
    base_loss.backward()
    optim_large_step.step()
    large_step_dict = deepcopy(network.state_dict())
    
    # multiple small steps
    network.load_state_dict(original_dict)
    optim_small_step = torch.optim.SGD(network.parameters(), lr=0.025/3)
    for i in range(3):
        optim_small_step.zero_grad()
        _, logits = network(inputs)
        base_loss = criterion(logits, targets)
        base_loss.backward()
        optim_small_step.step()
    small_step_dict = deepcopy(network.state_dict())
    scores = []
    for key in large_step_dict.keys():
        if ('weight' in key) and (original_dict[key].dim()==4):
            if (original_dict[key] != large_step_dict[key]).sum():
                large_step = large_step_dict[key] - original_dict[key]
                small_step = small_step_dict[key] - original_dict[key]
                co, ci, kh, kw = large_step.size()
                large_step = large_step.view(co, -1)
                small_step = small_step.view(co, -1)
                score = torch.nn.functional.cosine_similarity(large_step, small_step, dim=1)
                score = score.mean().item() * 100 # in percent
                scores.append(score)
    if len(scores)==0:
        step_sim = 100
        raise RuntimeError
    else:
        step_sim = np.mean(scores)
    
    # resume
    network.load_state_dict(original_dict)
            
    return step_sim

In [6]:
# start training
start_time, search_time, epoch_time, total_epoch = (
    time.time(),
    AverageMeter(),
    AverageMeter(),
    config.epochs + config.warmup,
)

################# initialize
cells = []
for m in network.modules():
    if isinstance(m, SearchCell):
        cells.append(m)
num_cells = len(cells)
print("total number of nodes:{}".format(num_cells*xargs.max_nodes))
        
op_names = deepcopy(cells[0].op_names)
op_names_wo_none = deepcopy(op_names)
if "none" in op_names_wo_none:
    op_names_wo_none.remove("none")

genotypes = []
for i in range(1, xargs.max_nodes):
    xlist = []
    for j in range(i):
        node_str = "{:}<-{:}".format(i, j)
        if i-j==1:
            op_name = "skip_connect"
        else:
            op_name = "none"
        xlist.append((op_name, j))
    genotypes.append(tuple(xlist))
init_arch = Structure(genotypes)

for c in cells:
    c.arch_cache = init_arch

### gen possible connections of a target node
possible_connections = {}
for target_node_idx in range(1,xargs.max_nodes):
    possible_connections[target_node_idx] = list()
    xlists = []
    for src_node in range(target_node_idx):
        node_str = "{:}<-{:}".format(target_node_idx, src_node)
        # select possible ops
#         if target_node_idx - src_node == 1:
#             op_names_tmp = op_names_wo_none
#         else:
#             op_names_tmp = op_names
        op_names_tmp = op_names
            
        if len(xlists) == 0: # initial iteration
            for op_name in op_names_tmp:
                xlists.append([(op_name, src_node)])
        else:
            new_xlists = []
            for op_name in op_names_tmp:
                for xlist in xlists:
                    new_xlist = deepcopy(xlist)
                    new_xlist.append((op_name, src_node))
                    new_xlists.append(new_xlist)
            xlists = new_xlists
    for xlist in xlists:
        selected_ops = []
        for l in xlist:
            selected_ops.append(l[0])
        if sum(np.array(selected_ops) == "none") == len(selected_ops):
            continue
        possible_connections[target_node_idx].append(tuple(xlist))
    print("target_node:{}".format(target_node_idx), len(possible_connections[target_node_idx]))
        
### train while generating random architectures by mutating connections of a target node

for arch_loop in range(5):
    for target_cell_idx in range(num_cells):
        target_cell = cells[target_cell_idx]
        print("\n\n Searching with a cell #{}".format(target_cell_idx))
        ####
#         for m in target_cell.modules():
#             if hasattr(m, 'reset_parameters'):
#                 m.reset_parameters()
        ####
        ## training
        for ep in range(2):
            ###
            genotypes = []
            for n in range(1, xargs.max_nodes):
                genotypes.append(random.choice(possible_connections[n]))
            arch = Structure(genotypes)
            target_cell.arch_cache = arch
#             arch = target_cell.random_genotype(True)
            ###
            data_time, batch_time = AverageMeter(), AverageMeter()
            base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
            network.train()
            end = time.time()
            print_freq = 200
            for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(search_loader):
                ######### forward/backward/optim
                base_targets = base_targets.cuda(non_blocking=True)
                arch_targets = arch_targets.cuda(non_blocking=True)
                # measure data loading time
                data_time.update(time.time() - end)
                w_optimizer.zero_grad()
                _, logits = network(base_inputs)
                base_loss = criterion(logits, base_targets)
                base_loss.backward()
                nn.utils.clip_grad_norm_(network.parameters(), 5)
                w_optimizer.step()

                ######### logging
                base_prec1, base_prec5 = obtain_accuracy(logits.data, base_targets.data, topk=(1, 5))
                base_losses.update(base_loss.item(), base_inputs.size(0))
                base_top1.update(base_prec1.item(), base_inputs.size(0))
                base_top5.update(base_prec5.item(), base_inputs.size(0))
                batch_time.update(time.time() - end)
                end = time.time()
                if step % print_freq == 0 or step + 1 == len(search_loader):
                    Sstr = ("*Train* "+ time_string()+" Ep:{:} [{:03d}/{:03d}]".format(ep, step, len(search_loader)))
                    Tstr = "Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})".format(batch_time=batch_time, data_time=data_time)
                    Wstr = "Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]".format(loss=base_losses, top1=base_top1, top5=base_top5)
                    logger.log(Sstr + " " + Tstr + " " + Wstr)

            logger.log("Ep:{:} ends : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%".format(ep, base_losses.avg, base_top1.avg, base_top5.avg))
        ## evaluation
        network.train()
        archs, metric_accs = [], []
        loader_iter = iter(valid_loader)
        for search_iter in range(200):
            ###### random gen
            genotypes = []
            for n in range(1, xargs.max_nodes):
                genotypes.append(random.choice(possible_connections[n]))
            arch = Structure(genotypes)
            target_cell.arch_cache = arch
#             arch = target_cell.random_genotype(True)
            ###### measure metrics
            try:
                inputs, targets = next(loader_iter)
            except:
                loader_iter = iter(valid_loader)
                inputs, targets = next(loader_iter)
            inputs, targets = inputs.cuda(non_blocking=True), targets.cuda(non_blocking=True)
            valid_acc = acc_confidence_robustness_metrics(network, inputs, targets)
            archs.append(arch)
            metric_accs.append(valid_acc)
        rank_accs = stats.rankdata(metric_accs)
        rank_agg = rank_accs
#         l = len(rank_accs)
#         rank_agg = np.log(rank_accs/l)+np.log(rank_confidences/l)+np.log(rank_sensitivities/l)+np.log(rank_robustnesses/l)+np.log(rank_step_sims/l)
#             rank_agg = np.log(rank_accs/l)+np.log(rank_confidences/l)+np.log(rank_sensitivities/l)+np.log(rank_step_sims/l)
        best_idx = np.argmax(rank_agg)
        best_arch, best_acc = archs[best_idx], metric_accs[best_idx]
        logger.log("Found best op for target cell:{}".format(target_cell_idx))
        logger.log(": {:} with accuracy={:.2f}%".format(best_arch, best_acc))
        target_cell.arch_cache = best_arch
            
best_archs = []
for c in cells:
    best_archs.append(c.arch_cache)
    
torch.save({"model":search_model.state_dict(), "best_archs":best_archs}, os.path.join(xargs.save_dir, "output.pth"))

for m in search_model.modules():
    if isinstance(m, SearchCell):
        logger.log(m.arch_cache)

logger.close()

total number of nodes:60
target_node:1 4
target_node:2 24
target_node:3 124


 Searching with a cell #0
*Train* [2022-11-07 02:53:58] Ep:0 [000/391] Time 2.60 (2.60) Data 0.21 (0.21) Base [Loss 2.294 (2.294)  Prec@1 10.94 (10.94) Prec@5 57.81 (57.81)]
*Train* [2022-11-07 02:54:22] Ep:0 [200/391] Time 0.08 (0.13) Data 0.00 (0.00) Base [Loss 1.594 (1.825)  Prec@1 39.06 (30.60) Prec@5 89.06 (84.26)]
*Train* [2022-11-07 02:54:40] Ep:0 [390/391] Time 0.13 (0.11) Data 0.00 (0.00) Base [Loss 1.753 (1.674)  Prec@1 35.00 (36.98) Prec@5 90.00 (87.82)]
Ep:0 ends : loss=1.67, accuracy@1=36.98%, accuracy@5=87.82%
*Train* [2022-11-07 02:54:41] Ep:1 [000/391] Time 0.30 (0.30) Data 0.21 (0.21) Base [Loss 2.792 (2.792)  Prec@1 10.94 (10.94) Prec@5 59.38 (59.38)]
*Train* [2022-11-07 02:55:00] Ep:1 [200/391] Time 0.20 (0.10) Data 0.00 (0.00) Base [Loss 1.544 (1.549)  Prec@1 45.31 (42.45) Prec@5 89.06 (90.76)]
*Train* [2022-11-07 02:55:25] Ep:1 [390/391] Time 0.12 (0.11) Data 0.00 (0.00) Base [Loss 1.470 

Found best op for target cell:6
: Structure(4 nodes with |skip_connect~0|+|avg_pool_3x3~0|nor_conv_3x3~1|+|skip_connect~0|none~1|none~2|) with accuracy=72.46%


 Searching with a cell #7
*Train* [2022-11-07 03:05:50] Ep:0 [000/391] Time 0.41 (0.41) Data 0.27 (0.27) Base [Loss 0.709 (0.709)  Prec@1 76.56 (76.56) Prec@5 95.31 (95.31)]
*Train* [2022-11-07 03:06:12] Ep:0 [200/391] Time 0.20 (0.11) Data 0.00 (0.00) Base [Loss 0.752 (0.774)  Prec@1 68.75 (73.06) Prec@5 98.44 (98.25)]
*Train* [2022-11-07 03:06:37] Ep:0 [390/391] Time 0.12 (0.12) Data 0.00 (0.00) Base [Loss 0.502 (0.760)  Prec@1 82.50 (73.43) Prec@5 100.00 (98.28)]
Ep:0 ends : loss=0.76, accuracy@1=73.43%, accuracy@5=98.28%
*Train* [2022-11-07 03:06:38] Ep:1 [000/391] Time 0.39 (0.39) Data 0.25 (0.25) Base [Loss 0.937 (0.937)  Prec@1 65.62 (65.62) Prec@5 98.44 (98.44)]
*Train* [2022-11-07 03:07:01] Ep:1 [200/391] Time 0.15 (0.12) Data 0.00 (0.00) Base [Loss 0.886 (0.736)  Prec@1 70.31 (74.05) Prec@5 95.31 (98.47)]
*Train* [202

*Train* [2022-11-07 03:17:49] Ep:1 [390/391] Time 0.07 (0.11) Data 0.00 (0.00) Base [Loss 0.450 (0.673)  Prec@1 87.50 (76.81) Prec@5 97.50 (98.49)]
Ep:1 ends : loss=0.67, accuracy@1=76.81%, accuracy@5=98.49%
Found best op for target cell:13
: Structure(4 nodes with |avg_pool_3x3~0|+|nor_conv_1x1~0|skip_connect~1|+|avg_pool_3x3~0|nor_conv_3x3~1|nor_conv_1x1~2|) with accuracy=77.54%


 Searching with a cell #14
*Train* [2022-11-07 03:18:10] Ep:0 [000/391] Time 0.41 (0.41) Data 0.23 (0.23) Base [Loss 0.771 (0.771)  Prec@1 68.75 (68.75) Prec@5 100.00 (100.00)]
*Train* [2022-11-07 03:18:30] Ep:0 [200/391] Time 0.12 (0.10) Data 0.00 (0.00) Base [Loss 0.789 (0.630)  Prec@1 71.88 (78.09) Prec@5 100.00 (98.87)]
*Train* [2022-11-07 03:18:55] Ep:0 [390/391] Time 0.14 (0.12) Data 0.00 (0.00) Base [Loss 0.483 (0.631)  Prec@1 77.50 (78.25) Prec@5 100.00 (98.82)]
Ep:0 ends : loss=0.63, accuracy@1=78.25%, accuracy@5=98.82%
*Train* [2022-11-07 03:18:56] Ep:1 [000/391] Time 0.42 (0.42) Data 0.26 (0.26) 

*Train* [2022-11-07 03:29:56] Ep:1 [200/391] Time 0.07 (0.13) Data 0.00 (0.00) Base [Loss 0.676 (0.915)  Prec@1 76.56 (67.82) Prec@5 100.00 (96.98)]
*Train* [2022-11-07 03:30:18] Ep:1 [390/391] Time 0.14 (0.12) Data 0.00 (0.00) Base [Loss 0.867 (0.825)  Prec@1 67.50 (71.28) Prec@5 97.50 (97.58)]
Ep:1 ends : loss=0.83, accuracy@1=71.28%, accuracy@5=97.58%
Found best op for target cell:5
: Structure(4 nodes with |nor_conv_3x3~0|+|none~0|avg_pool_3x3~1|+|none~0|skip_connect~1|none~2|) with accuracy=67.77%


 Searching with a cell #6
*Train* [2022-11-07 03:30:36] Ep:0 [000/391] Time 0.41 (0.41) Data 0.24 (0.24) Base [Loss 3.065 (3.065)  Prec@1 12.50 (12.50) Prec@5 62.50 (62.50)]
*Train* [2022-11-07 03:30:53] Ep:0 [200/391] Time 0.09 (0.09) Data 0.00 (0.00) Base [Loss 0.840 (0.825)  Prec@1 71.88 (71.38) Prec@5 100.00 (97.61)]
*Train* [2022-11-07 03:31:18] Ep:0 [390/391] Time 0.09 (0.11) Data 0.00 (0.00) Base [Loss 0.752 (0.778)  Prec@1 70.00 (72.97) Prec@5 100.00 (97.99)]
Ep:0 ends : loss=0

*Train* [2022-11-07 03:41:53] Ep:1 [000/391] Time 0.41 (0.41) Data 0.25 (0.25) Base [Loss 0.600 (0.600)  Prec@1 79.69 (79.69) Prec@5 100.00 (100.00)]
*Train* [2022-11-07 03:42:14] Ep:1 [200/391] Time 0.16 (0.11) Data 0.00 (0.00) Base [Loss 0.831 (0.601)  Prec@1 67.19 (79.32) Prec@5 98.44 (98.89)]
*Train* [2022-11-07 03:42:40] Ep:1 [390/391] Time 0.22 (0.12) Data 0.00 (0.00) Base [Loss 0.563 (0.599)  Prec@1 87.50 (79.52) Prec@5 97.50 (98.76)]
Ep:1 ends : loss=0.60, accuracy@1=79.52%, accuracy@5=98.76%
Found best op for target cell:12
: Structure(4 nodes with |avg_pool_3x3~0|+|none~0|avg_pool_3x3~1|+|skip_connect~0|skip_connect~1|none~2|) with accuracy=77.54%


 Searching with a cell #13
*Train* [2022-11-07 03:43:00] Ep:0 [000/391] Time 0.38 (0.38) Data 0.23 (0.23) Base [Loss 2.657 (2.657)  Prec@1 6.25 (6.25) Prec@5 57.81 (57.81)]
*Train* [2022-11-07 03:43:20] Ep:0 [200/391] Time 0.07 (0.10) Data 0.00 (0.00) Base [Loss 0.307 (0.648)  Prec@1 93.75 (78.28) Prec@5 100.00 (98.21)]
*Train* [2

...

*Train* [2022-11-07 04:06:25] Ep:0 [200/391] Time 0.07 (0.09) Data 0.00 (0.00) Base [Loss 1.097 (0.582)  Prec@1 57.81 (79.94) Prec@5 100.00 (99.04)]
*Train* [2022-11-07 04:06:42] Ep:0 [390/391] Time 0.08 (0.09) Data 0.00 (0.00) Base [Loss 0.422 (0.574)  Prec@1 92.50 (80.37) Prec@5 100.00 (99.01)]
Ep:0 ends : loss=0.57, accuracy@1=80.37%, accuracy@5=99.01%
*Train* [2022-11-07 04:06:43] Ep:1 [000/391] Time 0.39 (0.39) Data 0.23 (0.23) Base [Loss 0.604 (0.604)  Prec@1 78.12 (78.12) Prec@5 98.44 (98.44)]
*Train* [2022-11-07 04:07:04] Ep:1 [200/391] Time 0.09 (0.11) Data 0.00 (0.00) Base [Loss 0.383 (0.546)  Prec@1 84.38 (81.05) Prec@5 100.00 (99.21)]
*Train* [2022-11-07 04:07:24] Ep:1 [390/391] Time 0.14 (0.11) Data 0.00 (0.00) Base [Loss 0.579 (0.554)  Prec@1 82.50 (80.76) Prec@5 97.50 (99.16)]
Ep:1 ends : loss=0.55, accuracy@1=80.76%, accuracy@5=99.16%
Found best op for target cell:11
: Structure(4 nodes with |nor_conv_1x1~0|+|avg_pool_3x3~0|nor_conv_3x3~1|+|none~0|skip_connect~1|skip_co

*Train* [2022-11-07 04:18:22] Ep:0 [000/391] Time 0.33 (0.33) Data 0.23 (0.23) Base [Loss 2.848 (2.848)  Prec@1 20.31 (20.31) Prec@5 73.44 (73.44)]
*Train* [2022-11-07 04:18:40] Ep:0 [200/391] Time 0.09 (0.09) Data 0.00 (0.00) Base [Loss 0.595 (0.868)  Prec@1 82.81 (70.02) Prec@5 98.44 (97.43)]
*Train* [2022-11-07 04:19:04] Ep:0 [390/391] Time 0.09 (0.11) Data 0.00 (0.00) Base [Loss 0.730 (0.815)  Prec@1 72.50 (71.62) Prec@5 97.50 (97.84)]
Ep:0 ends : loss=0.82, accuracy@1=71.62%, accuracy@5=97.84%
*Train* [2022-11-07 04:19:04] Ep:1 [000/391] Time 0.38 (0.38) Data 0.27 (0.27) Base [Loss 0.507 (0.507)  Prec@1 82.81 (82.81) Prec@5 100.00 (100.00)]
*Train* [2022-11-07 04:19:22] Ep:1 [200/391] Time 0.08 (0.09) Data 0.00 (0.00) Base [Loss 0.666 (0.696)  Prec@1 78.12 (75.75) Prec@5 100.00 (98.33)]
*Train* [2022-11-07 04:19:42] Ep:1 [390/391] Time 0.13 (0.10) Data 0.00 (0.00) Base [Loss 0.648 (0.684)  Prec@1 77.50 (76.21) Prec@5 97.50 (98.52)]
Ep:1 ends : loss=0.68, accuracy@1=76.21%, accurac

Found best op for target cell:9
: Structure(4 nodes with |skip_connect~0|+|avg_pool_3x3~0|none~1|+|skip_connect~0|none~1|skip_connect~2|) with accuracy=74.02%


 Searching with a cell #10
*Train* [2022-11-07 04:31:08] Ep:0 [000/391] Time 0.37 (0.37) Data 0.23 (0.23) Base [Loss 0.703 (0.703)  Prec@1 75.00 (75.00) Prec@5 100.00 (100.00)]
*Train* [2022-11-07 04:31:30] Ep:0 [200/391] Time 0.09 (0.11) Data 0.00 (0.00) Base [Loss 0.406 (0.677)  Prec@1 89.06 (76.47) Prec@5 100.00 (98.45)]
*Train* [2022-11-07 04:31:51] Ep:0 [390/391] Time 0.14 (0.11) Data 0.00 (0.00) Base [Loss 0.812 (0.664)  Prec@1 72.50 (76.78) Prec@5 97.50 (98.61)]
Ep:0 ends : loss=0.66, accuracy@1=76.78%, accuracy@5=98.61%
*Train* [2022-11-07 04:31:51] Ep:1 [000/391] Time 0.35 (0.35) Data 0.26 (0.26) Base [Loss 0.981 (0.981)  Prec@1 67.19 (67.19) Prec@5 98.44 (98.44)]
*Train* [2022-11-07 04:32:15] Ep:1 [200/391] Time 0.14 (0.12) Data 0.00 (0.00) Base [Loss 0.642 (0.654)  Prec@1 81.25 (77.29) Prec@5 100.00 (98.48)]
*Train* 

*Train* [2022-11-07 04:43:53] Ep:1 [390/391] Time 0.07 (0.11) Data 0.00 (0.00) Base [Loss 0.821 (0.601)  Prec@1 65.00 (79.37) Prec@5 97.50 (98.77)]
Ep:1 ends : loss=0.60, accuracy@1=79.37%, accuracy@5=98.77%
Found best op for target cell:1
: Structure(4 nodes with |skip_connect~0|+|skip_connect~0|skip_connect~1|+|nor_conv_3x3~0|avg_pool_3x3~1|none~2|) with accuracy=81.25%


 Searching with a cell #2
*Train* [2022-11-07 04:44:14] Ep:0 [000/391] Time 0.51 (0.51) Data 0.25 (0.25) Base [Loss 0.722 (0.722)  Prec@1 78.12 (78.12) Prec@5 96.88 (96.88)]
*Train* [2022-11-07 04:44:36] Ep:0 [200/391] Time 0.14 (0.11) Data 0.00 (0.00) Base [Loss 0.554 (0.517)  Prec@1 78.12 (82.07) Prec@5 100.00 (99.14)]
*Train* [2022-11-07 04:44:55] Ep:0 [390/391] Time 0.08 (0.11) Data 0.00 (0.00) Base [Loss 0.430 (0.516)  Prec@1 87.50 (82.02) Prec@5 100.00 (99.16)]
Ep:0 ends : loss=0.52, accuracy@1=82.02%, accuracy@5=99.16%
*Train* [2022-11-07 04:44:56] Ep:1 [000/391] Time 0.44 (0.44) Data 0.28 (0.28) Base [Loss 0

*Train* [2022-11-07 04:56:17] Ep:1 [200/391] Time 0.14 (0.10) Data 0.00 (0.00) Base [Loss 0.743 (0.620)  Prec@1 76.56 (78.46) Prec@5 96.88 (98.85)]
*Train* [2022-11-07 04:56:45] Ep:1 [390/391] Time 0.09 (0.12) Data 0.00 (0.00) Base [Loss 0.664 (0.601)  Prec@1 80.00 (79.33) Prec@5 97.50 (98.87)]
Ep:1 ends : loss=0.60, accuracy@1=79.33%, accuracy@5=98.87%
Found best op for target cell:8
: Structure(4 nodes with |nor_conv_1x1~0|+|skip_connect~0|nor_conv_3x3~1|+|skip_connect~0|nor_conv_1x1~1|nor_conv_1x1~2|) with accuracy=74.61%


 Searching with a cell #9
*Train* [2022-11-07 04:57:05] Ep:0 [000/391] Time 0.34 (0.34) Data 0.23 (0.23) Base [Loss 0.894 (0.894)  Prec@1 65.62 (65.62) Prec@5 98.44 (98.44)]
*Train* [2022-11-07 04:57:26] Ep:0 [200/391] Time 0.09 (0.11) Data 0.00 (0.00) Base [Loss 0.675 (0.570)  Prec@1 71.88 (79.84) Prec@5 100.00 (99.23)]
*Train* [2022-11-07 04:57:46] Ep:0 [390/391] Time 0.12 (0.11) Data 0.00 (0.00) Base [Loss 0.648 (0.560)  Prec@1 77.50 (80.36) Prec@5 100.00 (99.

In [7]:
# import matplotlib.pyplot as plt

# plt.scatter(rank_confidences,rank_accs)
# plt.show()

# plt.scatter(rank_sensitivities,rank_accs)
# plt.show()

# plt.scatter(rank_robustnesses,rank_accs)
# plt.show()

# plt.scatter(rank_step_sims,rank_accs)
# plt.show()

# Train a found model

In [8]:
trained_output = torch.load(os.path.join(xargs.save_dir, "output.pth"))
print(args)
args.save_dir = os.path.join(xargs.save_dir, "train")
print(args)

Namespace(arch_nas_dataset=None, channel=16, config_path='./MY.config', data_path='../cifar.python', dataset='cifar10', max_nodes=4, num_cells=5, print_freq=200, rand_seed=65819, save_dir='./cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric', search_space_name='nas-bench-201', select_num=100, track_running_stats=0, workers=4)
Namespace(arch_nas_dataset=None, channel=16, config_path='./MY.config', data_path='../cifar.python', dataset='cifar10', max_nodes=4, num_cells=5, print_freq=200, rand_seed=65819, save_dir='./cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train', search_space_name='nas-bench-201', select_num=100, track_running_stats=0, workers=4)


In [9]:
print(config)

Configure(scheduler='cos', LR=0.025, eta_min=0.001, epochs=50, warmup=0, optim='SGD', decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=64, test_batch_size=512, class_num=10, xshape=(1, 3, 32, 32))


In [10]:
logger = prepare_logger(args)

# cifar_train_config_path = "./MY.config"
cifar_train_config_path = "../configs/nas-benchmark/CIFAR.config"
###
train_data, test_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
config = load_config(cifar_train_config_path, {"class_num": class_num, "xshape": xshape}, logger)

train_loader = torch.utils.data.DataLoader(
            train_data,
            batch_size=config.batch_size,
            shuffle=True,
            num_workers=xargs.workers,
            pin_memory=True,)

test_loader = torch.utils.data.DataLoader(
            test_data,
            batch_size=config.batch_size,
            shuffle=True,
            num_workers=xargs.workers,
            pin_memory=True,)

# search_loader, _, valid_loader = get_nas_search_loaders(train_data,
#                                                         valid_data,
#                                                         xargs.dataset,
#                                                         "../configs/nas-benchmark/",
#                                                         (config.batch_size, config.batch_size),
#                                                         xargs.workers)
logger.log("||||||| {:10s} ||||||| Train-Loader-Num={:}, Test-Loader-Num={:}, batch size={:}".format(
            xargs.dataset, len(train_loader), len(test_loader), config.batch_size))
logger.log("||||||| {:10s} ||||||| Config={:}".format(xargs.dataset, config))

search_space = get_search_spaces("cell", xargs.search_space_name)
model_config = dict2config(
    {
        "name": "RANDOM",
        "C": xargs.channel,
        "N": xargs.num_cells,
        "max_nodes": xargs.max_nodes,
        "num_classes": class_num,
        "space": search_space,
        "affine": False,
        "track_running_stats": True, # true for eval
    },
    None,
)
search_model = get_cell_based_tiny_net(model_config)

### load
# trained_output = torch.load(os.path.join(xargs.save_dir, "output.pth"))
# search_model.load_state_dict(trained_output['model'], strict=False)
best_archs = trained_output['best_archs']
i=0
for m in search_model.modules():
    if isinstance(m, SearchCell):
        m.arch_cache = best_archs[i]
        i += 1
for m in network.modules():
    if isinstance(m, SearchCell):
        print(m.arch_cache)
###

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.parameters(), config)

logger.log("w-optimizer : {:}".format(w_optimizer))
logger.log("w-scheduler : {:}".format(w_scheduler))
logger.log("criterion   : {:}".format(criterion))

network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()

last_info, model_base_path, model_best_path = (
    logger.path("info"),
    logger.path("model"),
    logger.path("best"),
)

start_epoch, valid_accuracies, genotypes = 0, {"best": -1}, {}

Main Function with logger : Logger(dir=cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train, use-tf=False, writer=None)
Arguments : -------------------------------
arch_nas_dataset : None
channel          : 16
config_path      : ./MY.config
data_path        : ../cifar.python
dataset          : cifar10
max_nodes        : 4
num_cells        : 5
print_freq       : 200
rand_seed        : 65819
save_dir         : ./cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train
search_space_name : nas-bench-201
select_num       : 100
track_running_stats : 0
workers          : 4
Python  Version  : 3.7.13 (default, Mar 29 2022, 02:18:16)  [GCC 7.5.0]
Pillow  Version  : 9.0.1
PyTorch Version  : 1.12.0
cuDNN   Version  : 8302
CUDA available   : True
CUDA GPU numbers : 2
CUDA_VISIBLE_DEVICES : None
Files already downloaded and verified
Files already downloaded and verified
../configs/nas-benchmark/CIFAR.config
Configure(scheduler='cos', eta_min=0.0, epoch

In [11]:
# def search_func_one_arch(xloader, network, criterion, scheduler, w_optimizer, epoch_str, print_freq, logger):
#     data_time, batch_time = AverageMeter(), AverageMeter()
#     base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
#     network.train()
#     end = time.time()
#     for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(
#         xloader
#     ):
#         scheduler.update(None, 1.0 * step / len(xloader))
#         base_targets = base_targets.cuda(non_blocking=True)
#         arch_targets = arch_targets.cuda(non_blocking=True)
#         # measure data loading time
#         data_time.update(time.time() - end)

#         w_optimizer.zero_grad()
#         _, logits = network(base_inputs)
#         base_loss = criterion(logits, base_targets)
#         base_loss.backward()
#         nn.utils.clip_grad_norm_(network.parameters(), 5)
#         w_optimizer.step()
#         # record
#         base_prec1, base_prec5 = obtain_accuracy(
#             logits.data, base_targets.data, topk=(1, 5)
#         )
#         base_losses.update(base_loss.item(), base_inputs.size(0))
#         base_top1.update(base_prec1.item(), base_inputs.size(0))
#         base_top5.update(base_prec5.item(), base_inputs.size(0))

#         # measure elapsed time
#         batch_time.update(time.time() - end)
#         end = time.time()

#         if step % print_freq == 0 or step + 1 == len(xloader):
#             Sstr = (
#                 "*SEARCH* "
#                 + time_string()
#                 + " [{:}][{:03d}/{:03d}]".format(epoch_str, step, len(xloader))
#             )
#             Tstr = "Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})".format(
#                 batch_time=batch_time, data_time=data_time
#             )
#             Wstr = "Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]".format(
#                 loss=base_losses, top1=base_top1, top5=base_top5
#             )
#             logger.log(Sstr + " " + Tstr + " " + Wstr)
#     return base_losses.avg, base_top1.avg, base_top5.avg

def train_func_one_arch(xloader, network, criterion, scheduler, w_optimizer, epoch_str, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()
    for step, (base_inputs, base_targets) in enumerate(
        xloader
    ):
        scheduler.update(None, 1.0 * step / len(xloader))
        base_targets = base_targets.cuda(non_blocking=True)
        # measure data loading time
        data_time.update(time.time() - end)

        w_optimizer.zero_grad()
        _, logits = network(base_inputs)
        base_loss = criterion(logits, base_targets)
        base_loss.backward()
        nn.utils.clip_grad_norm_(network.parameters(), 5)
        w_optimizer.step()
        # record
        base_prec1, base_prec5 = obtain_accuracy(
            logits.data, base_targets.data, topk=(1, 5)
        )
        base_losses.update(base_loss.item(), base_inputs.size(0))
        base_top1.update(base_prec1.item(), base_inputs.size(0))
        base_top5.update(base_prec5.item(), base_inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if step % print_freq == 0 or step + 1 == len(xloader):
            Sstr = (
                "*SEARCH* "
                + time_string()
                + " [{:}][{:03d}/{:03d}]".format(epoch_str, step, len(xloader))
            )
            Tstr = "Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})".format(
                batch_time=batch_time, data_time=data_time
            )
            Wstr = "Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]".format(
                loss=base_losses, top1=base_top1, top5=base_top5
            )
            logger.log(Sstr + " " + Tstr + " " + Wstr)
    return base_losses.avg, base_top1.avg, base_top5.avg

def valid_func_one_arch(xloader, network, criterion):
    data_time, batch_time = AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.eval()
    end = time.time()
    with torch.no_grad():
        for step, (arch_inputs, arch_targets) in enumerate(xloader):
            arch_targets = arch_targets.cuda(non_blocking=True)
            # measure data loading time
            data_time.update(time.time() - end)
            # prediction

#             network.module.random_genotype_per_cell(True)
            _, logits = network(arch_inputs)
            arch_loss = criterion(logits, arch_targets)
            # record
            arch_prec1, arch_prec5 = obtain_accuracy(
                logits.data, arch_targets.data, topk=(1, 5)
            )
            arch_losses.update(arch_loss.item(), arch_inputs.size(0))
            arch_top1.update(arch_prec1.item(), arch_inputs.size(0))
            arch_top5.update(arch_prec5.item(), arch_inputs.size(0))
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
    return arch_losses.avg, arch_top1.avg, arch_top5.avg

In [12]:
start_time, search_time, epoch_time, total_epoch = (
    time.time(),
    AverageMeter(),
    AverageMeter(),
    config.epochs + config.warmup,
)
for epoch in range(0, total_epoch):
    w_scheduler.update(epoch, 0.0)
    need_time = "Time Left: {:}".format(
        convert_secs2time(epoch_time.val * (total_epoch - epoch), True)
    )
    epoch_str = "{:03d}-{:03d}".format(epoch, total_epoch)
    logger.log(
        "\n[Search the {:}-th epoch] {:}, LR={:}".format(
            epoch_str, need_time, min(w_scheduler.get_lr())
        )
    )

    # selected_arch = search_find_best(valid_loader, network, criterion, xargs.select_num)
    search_w_loss, search_w_top1, search_w_top5 = train_func_one_arch(
        train_loader,
        network,
        criterion,
        w_scheduler,
        w_optimizer,
        epoch_str,
        xargs.print_freq,
        logger,
    )
    search_time.update(time.time() - start_time)
    logger.log(
        "[{:}] searching : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%, time-cost={:.1f} s".format(
            epoch_str, search_w_loss, search_w_top1, search_w_top5, search_time.sum
        )
    )
    valid_a_loss, valid_a_top1, valid_a_top5 = valid_func_one_arch(
        test_loader, network, criterion
    )
    logger.log(
        "[{:}] evaluate  : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%".format(
            epoch_str, valid_a_loss, valid_a_top1, valid_a_top5
        )
    )
    
    # check the best accuracy
    valid_accuracies[epoch] = valid_a_top1
    if valid_a_top1 > valid_accuracies["best"]:
        valid_accuracies["best"] = valid_a_top1
        find_best = True
    else:
        find_best = False

    # save checkpoint
    save_path = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(xargs),
            "search_model": search_model.state_dict(),
            "w_optimizer": w_optimizer.state_dict(),
            "w_scheduler": w_scheduler.state_dict(),
            "genotypes": genotypes,
            "valid_accuracies": valid_accuracies,
        },
        model_base_path,
        logger,
    )
    last_info = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(args),
            "last_checkpoint": save_path,
        },
        logger.path("info"),
        logger,
    )
    if find_best:
        logger.log(
            "<<<--->>> The {:}-th epoch : find the highest validation accuracy : {:.2f}%.".format(
                epoch_str, valid_a_top1
            )
        )
        copy_checkpoint(model_base_path, model_best_path, logger)
    if api is not None:
        logger.log("{:}".format(api.query_by_arch(genotypes[epoch], "200")))
    # measure elapsed time
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

logger.close()


[Search the 000-200-th epoch] Time Left: [00:00:00], LR=0.1
*SEARCH* [2022-11-07 05:08:29] [000-200][000/196] Time 0.38 (0.38) Data 0.26 (0.26) Base [Loss 2.321 (2.321)  Prec@1 9.77 (9.77) Prec@5 55.47 (55.47)]
*SEARCH* [2022-11-07 05:08:54] [000-200][195/196] Time 0.09 (0.13) Data 0.00 (0.00) Base [Loss 1.375 (1.673)  Prec@1 51.25 (37.69) Prec@5 93.75 (87.40)]
[000-200] searching : loss=1.67, accuracy@1=37.69%, accuracy@5=87.40%, time-cost=24.7 s
[000-200] evaluate  : loss=4.32, accuracy@1=17.97%, accuracy@5=73.91%
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/seed-65819-last-info.pth
<<<--->>> The 000-200-th epoch : find the highest validation accuracy : 17.97%.
copy the file from cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth into cell_

*SEARCH* [2022-11-07 05:12:00] [006-200][195/196] Time 0.08 (0.13) Data 0.00 (0.00) Base [Loss 0.633 (0.673)  Prec@1 76.25 (76.52) Prec@5 98.75 (98.64)]
[006-200] searching : loss=0.67, accuracy@1=76.52%, accuracy@5=98.64%, time-cost=186.1 s
[006-200] evaluate  : loss=1.36, accuracy@1=58.98%, accuracy@5=96.82%
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/seed-65819-last-info.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/seed-65819-last-info.pth

[Search the 007-200-th epoch] Time Left: [01:30:11], LR=0.099698047772759
*SEARCH* [2022-11-07 05:12:03] [007-200][000/196] T

*SEARCH* [2022-11-07 05:15:02] [013-200][000/196] Time 0.37 (0.37) Data 0.25 (0.25) Base [Loss 0.495 (0.495)  Prec@1 82.03 (82.03) Prec@5 99.61 (99.61)]
*SEARCH* [2022-11-07 05:15:27] [013-200][195/196] Time 0.07 (0.13) Data 0.00 (0.00) Base [Loss 0.718 (0.537)  Prec@1 73.75 (81.49) Prec@5 97.50 (99.14)]
[013-200] searching : loss=0.54, accuracy@1=81.49%, accuracy@5=99.14%, time-cost=369.5 s
[013-200] evaluate  : loss=1.10, accuracy@1=66.07%, accuracy@5=96.55%
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/seed-65819-last-info.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/trai

*SEARCH* [2022-11-07 05:18:22] [020-200][000/196] Time 0.45 (0.45) Data 0.24 (0.24) Base [Loss 0.438 (0.438)  Prec@1 83.59 (83.59) Prec@5 99.22 (99.22)]
*SEARCH* [2022-11-07 05:18:51] [020-200][195/196] Time 0.15 (0.15) Data 0.00 (0.00) Base [Loss 0.382 (0.484)  Prec@1 86.25 (83.22) Prec@5 100.00 (99.27)]
[020-200] searching : loss=0.48, accuracy@1=83.22%, accuracy@5=99.27%, time-cost=549.4 s
[020-200] evaluate  : loss=1.02, accuracy@1=67.28%, accuracy@5=97.41%
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/seed-65819-last-info.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/tra

*SEARCH* [2022-11-07 05:22:02] [027-200][000/196] Time 0.40 (0.40) Data 0.28 (0.28) Base [Loss 0.434 (0.434)  Prec@1 83.98 (83.98) Prec@5 99.61 (99.61)]
*SEARCH* [2022-11-07 05:22:30] [027-200][195/196] Time 0.10 (0.14) Data 0.00 (0.00) Base [Loss 0.402 (0.445)  Prec@1 87.50 (84.77) Prec@5 98.75 (99.41)]
[027-200] searching : loss=0.45, accuracy@1=84.77%, accuracy@5=99.41%, time-cost=743.1 s
[027-200] evaluate  : loss=0.98, accuracy@1=69.72%, accuracy@5=97.80%
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/seed-65819-last-info.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/trai

...

*SEARCH* [2022-11-07 06:41:58] [167-200][000/196] Time 0.44 (0.44) Data 0.31 (0.31) Base [Loss 0.074 (0.074)  Prec@1 98.05 (98.05) Prec@5 100.00 (100.00)]
*SEARCH* [2022-11-07 06:42:29] [167-200][195/196] Time 0.15 (0.16) Data 0.00 (0.00) Base [Loss 0.041 (0.055)  Prec@1 98.75 (98.21) Prec@5 100.00 (99.99)]
[167-200] searching : loss=0.06, accuracy@1=98.21%, accuracy@5=99.99%, time-cost=4998.4 s
[167-200] evaluate  : loss=0.35, accuracy@1=90.21%, accuracy@5=99.61%
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/seed-65819-last-info.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/

*SEARCH* [2022-11-07 06:46:02] [174-200][000/196] Time 0.57 (0.57) Data 0.27 (0.27) Base [Loss 0.025 (0.025)  Prec@1 99.22 (99.22) Prec@5 100.00 (100.00)]
*SEARCH* [2022-11-07 06:46:32] [174-200][195/196] Time 0.14 (0.16) Data 0.00 (0.00) Base [Loss 0.079 (0.038)  Prec@1 98.75 (98.81) Prec@5 100.00 (100.00)]
[174-200] searching : loss=0.04, accuracy@1=98.81%, accuracy@5=100.00%, time-cost=5214.0 s
[174-200] evaluate  : loss=0.32, accuracy@1=91.48%, accuracy@5=99.73%
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/seed-65819-last-info.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metri

*SEARCH* [2022-11-07 06:49:58] [181-200][000/196] Time 0.45 (0.45) Data 0.31 (0.31) Base [Loss 0.012 (0.012)  Prec@1 100.00 (100.00) Prec@5 100.00 (100.00)]
*SEARCH* [2022-11-07 06:50:29] [181-200][195/196] Time 0.10 (0.16) Data 0.00 (0.00) Base [Loss 0.024 (0.025)  Prec@1 100.00 (99.32) Prec@5 100.00 (100.00)]
[181-200] searching : loss=0.02, accuracy@1=99.32%, accuracy@5=100.00%, time-cost=5424.4 s
[181-200] evaluate  : loss=0.34, accuracy@1=91.25%, accuracy@5=99.69%
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/seed-65819-last-info.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_me

*SEARCH* [2022-11-07 06:53:54] [188-200][000/196] Time 0.36 (0.36) Data 0.25 (0.25) Base [Loss 0.011 (0.011)  Prec@1 100.00 (100.00) Prec@5 100.00 (100.00)]
*SEARCH* [2022-11-07 06:54:25] [188-200][195/196] Time 0.15 (0.16) Data 0.00 (0.00) Base [Loss 0.023 (0.018)  Prec@1 100.00 (99.62) Prec@5 100.00 (100.00)]
[188-200] searching : loss=0.02, accuracy@1=99.62%, accuracy@5=100.00%, time-cost=5635.3 s
[188-200] evaluate  : loss=0.33, accuracy@1=91.54%, accuracy@5=99.72%
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/seed-65819-last-info.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_me

[195-200] evaluate  : loss=0.33, accuracy@1=91.52%, accuracy@5=99.72%
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/checkpoint/seed-65819-basic.pth
Find cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/seed-65819-last-info.pth exist, delete is at first before saving
save checkpoint into cell_level-arch_loop-no_reset_cell_params-loop5_ep2_sample200-acc_metric/train/seed-65819-last-info.pth

[Search the 196-200-th epoch] Time Left: [00:02:19], LR=9.866357858642206e-05
*SEARCH* [2022-11-07 06:58:27] [196-200][000/196] Time 0.52 (0.52) Data 0.34 (0.34) Base [Loss 0.023 (0.023)  Prec@1 98.83 (98.83) Prec@5 100.00 (100.00)]
*SEARCH* [2022-11-07 06:58:57] [196-200][195/196] Time 0.09 (0.15) Data 0.00 (0.00) Base [Loss 0.048 (0.016)  Prec@1 97.50 (99.68) Pre

In [13]:
best_archs

[Structure(4 nodes with |avg_pool_3x3~0|+|none~0|avg_pool_3x3~1|+|nor_conv_3x3~0|skip_connect~1|skip_connect~2|),
 Structure(4 nodes with |skip_connect~0|+|skip_connect~0|skip_connect~1|+|nor_conv_3x3~0|avg_pool_3x3~1|none~2|),
 Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_1x1~0|nor_conv_1x1~1|+|skip_connect~0|avg_pool_3x3~1|nor_conv_3x3~2|),
 Structure(4 nodes with |skip_connect~0|+|avg_pool_3x3~0|nor_conv_1x1~1|+|skip_connect~0|nor_conv_1x1~1|nor_conv_1x1~2|),
 Structure(4 nodes with |skip_connect~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|nor_conv_3x3~0|nor_conv_1x1~1|avg_pool_3x3~2|),
 Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|nor_conv_1x1~1|+|nor_conv_1x1~0|skip_connect~1|none~2|),
 Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_1x1~0|nor_conv_3x3~1|+|skip_connect~0|skip_connect~1|skip_connect~2|),
 Structure(4 nodes with |skip_connect~0|+|none~0|avg_pool_3x3~1|+|nor_conv_3x3~0|avg_pool_3x3~1|nor_conv_1x1~2|),
 Structure(4 nodes with |nor_conv_1x1~0|+|skip_connect~0